## Create a simple assistant using an LLM and Pydantic

Given a product name, it should return:

```python
Product --> {
  product_name: str,
  product_details: str,
  tentative_price: int | float  # in USD
}

Use ChatPromptTemplate

In [1]:
import os
import json
from typing import Union, List
from pydantic import BaseModel, Field
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser


In [2]:
# Set the Groq API Key
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

# Langsmith tracing and monitoring
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT") 

In [3]:
# Define Pydantic output schema
class ProductInfo(BaseModel):
    product_name: str = Field(..., description="The name of the product")
    product_details: str = Field(..., description="A brief description of the product")
    tentative_price: Union[float, int] = Field(..., description="The estimated price in USD")


In [4]:
# Create an output parser based on Pydantic
output_parser = PydanticOutputParser(pydantic_object=ProductInfo)

output_parser


PydanticOutputParser(pydantic_object=<class '__main__.ProductInfo'>)

In [5]:
# Create the prompt
prompt = ChatPromptTemplate.from_template(
    """
You are a helpful product assistant. Given a product name, return a JSON object with:
- product_name
- product_details
- tentative_price ((a number only)


Do not include any currency symbols. Just return a number.

Respond only with valid JSON that matches this format.

Product: {product_name}
"""
)

prompt

ChatPromptTemplate(input_variables=['product_name'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['product_name'], input_types={}, partial_variables={}, template='\nYou are a helpful product assistant. Given a product name, return a JSON object with:\n- product_name\n- product_details\n- tentative_price ((a number only)\n\n\nDo not include any currency symbols. Just return a number.\n\nRespond only with valid JSON that matches this format.\n\nProduct: {product_name}\n'), additional_kwargs={})])

In [6]:
# Load the Groq model
model = ChatGroq(
    model="gemma2-9b-it",  
    temperature=0 # use a low temerature to get somewhat consistent resoonses 
)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x106931040>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x10844ef90>, model_name='gemma2-9b-it', temperature=1e-08, model_kwargs={}, groq_api_key=SecretStr('**********'))

In [7]:
# Chain: prompt | model | parser
chain = prompt | model | output_parser
chain

ChatPromptTemplate(input_variables=['product_name'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['product_name'], input_types={}, partial_variables={}, template='\nYou are a helpful product assistant. Given a product name, return a JSON object with:\n- product_name\n- product_details\n- tentative_price ((a number only)\n\n\nDo not include any currency symbols. Just return a number.\n\nRespond only with valid JSON that matches this format.\n\nProduct: {product_name}\n'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x106931040>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x10844ef90>, model_name='gemma2-9b-it', temperature=1e-08, model_kwargs={}, groq_api_key=SecretStr('**********'))
| PydanticOutputParser(pydantic_object=<class '__main__.ProductInfo'>)

In [8]:
# Function to get product info
def get_product_info(product_name: str) -> ProductInfo:
    return chain.invoke({"product_name": product_name})

In [ ]:
def display_product_info(info: ProductInfo) -> str:
    # Convert to dict
    data = info.model_dump()
    
    # Format the price as a USD string
    data["tentative_price"] = f"${data['tentative_price']:,.2f} USD"
    
    # Pretty-print JSON string
    json_str = json.dumps(data, indent=2)
    print(json_str)
    return json_str



In [13]:
# Test on a single product
product = "Sony WH-1000XM5"
try:
    info = get_product_info(product)
    display_product_info(info)
except Exception as e:
    print(f"Error: {e}")


{
  "product_name": "Sony WH-1000XM5",
  "product_details": "Wireless noise-canceling headphones with industry-leading audio quality, advanced features, and comfortable design.",
  "tentative_price": "$350.00 USD"
}


In [23]:
def run_batch(products: List[str]):
    all_infos = []
    for p in products:
        try:
            info = chain.invoke({"product_name": p})
            formatted = display_product_info(info)
            print(formatted)
            all_infos.append(formatted)
        except Exception as e:
            print(f"Error processing '{p}': {e}")
    return all_infos 


In [24]:
# Test on a list of test products
test_products = [
    "Sony WH-1000XM5",
    "Apple Watch Series 9",
    "Kindle Paperwhite",
    "Nintendo Switch OLED",
    "Samsung Galaxy Tab S9"]

results = run_batch(test_products)
print(json.dumps(results, indent=2))

{
  "product_name": "Sony WH-1000XM5",
  "product_details": "Wireless noise-canceling headphones with industry-leading audio quality, advanced features, and comfortable design.",
  "tentative_price": "$350.00 USD"
}
None
{
  "product_name": "Apple Watch Series 9",
  "product_details": "Latest smartwatch with advanced health and fitness tracking features, cellular connectivity, and a bright always-on display.",
  "tentative_price": "$399.00 USD"
}
None
{
  "product_name": "Kindle Paperwhite",
  "product_details": "A waterproof e-reader with a 6.8\" glare-free display, adjustable warm light, and weeks of battery life.",
  "tentative_price": "$140.00 USD"
}
None
{
  "product_name": "Nintendo Switch OLED",
  "product_details": "A premium version of the Nintendo Switch console with a larger, brighter OLED screen, enhanced audio, and a wider adjustable stand.",
  "tentative_price": "$350.00 USD"
}
None
{
  "product_name": "Samsung Galaxy Tab S9",
  "product_details": "Latest Android tablet w